<a href="https://colab.research.google.com/github/davidjcox333/Man-vs-Machine/blob/master/GME_fits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set path to data
from google.colab import drive
drive.mount('/content/gdrive')
%cd './gdrive/My Drive/Matching with RE288/Data'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Matching with RE288/Data


In [23]:
# Packages we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from google.colab import files
import zipfile, io, os
import math

# Fit GME for all pitchers for each season using RE288 as reinforcer. 

In [3]:
# Read in file
data_raw = pd.read_csv("All_MLB_data_for_GME.csv")
data_raw = data_raw.drop(['Unnamed: 0'], axis=1)
data = data_raw.copy()
data.head()

,pitch_type,game_date,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,fielder_2,effective_speed,game_pk,pitcher.1,fielder_2.1,at_bat_number,pitch_number,pitch_name,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,game_state,inning_same,total_runs,change_score,runs_inn_end,RE288,RE288_change
0,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,field_out,hit_into_play,5.0,Brock Holt lines out to left fielder Rajai Dav...,R,L,R,CLE,BOS,X,7.0,line_drive,3.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,93.704,446867.0,545333.0,543228.0,74.0,5.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,65.0,False,8,0,0.0,1.969061,-0.016539
1,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,2.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.288,446867.0,545333.0,543228.0,74.0,4.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,61.0,True,8,0,0.0,1.952521,0.015919
2,FT,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,12.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,1.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.067,446867.0,545333.0,543228.0,74.0,3.0,2-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,57.0,True,8,0,0.0,1.968441,0.002399
3,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,0.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.092,446867.0,545333.0,543228.0,74.0,2.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Strategic,53.0,True,8,0,0.0,1.970840,-0.030190
4,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,foul,9.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,0.0,0.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.887,446867.0,545333.0,543228.0,74.0,1.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,49.0,True,8,0,2.0,1.940650,0.012106


In [4]:
# Create a list of all the unique pitchers in the df
uniq_p = data.player_name.unique()

In [5]:
# Identify which pitches fall in which categories
hrd_strt = ['4-Seam Fastball', '2-Seam Fastball', 'Cutter']
hrd_brkng = ['Slider', 'Sinker', 'Split Finger', 'Screwball', 'Forkball']
sft_strt = ['Changeup', 'Knuckle Ball', 'Eephus']
sft_brkng = ['Curveball', 'Knuckle Curve']

In [19]:
# Strike and out categories 
s_o = ['hit_into_play', 'foul','swinging_strike', 'swinging_strike_blocked', \
       'called_strike','foul_bunt', 'foul_tip', 'missed_bunt','swinging_pitchout', \
       'pitchout_hit_into_play_score','foul_pitchout', 'bunt_foul_tip']

In [32]:
# Calculate GME for each pitcher in df using RE288 as outcome
# Make sure data types are as we need them
data.game_year = data.game_year.astype(int)

player = []
season = []
vAC_RE288 = []
bias_RE288 = []
sens_RE288 = []
VAC_strikes = []
bias_strikes = []
ens_strikes = []

# Pull df for the target pitcher and break down by season
for i in uniq_p[:1]:
  # Lists to store things in
  game_number = []
  year = []
  hard_straight_count = []
  hard_breaking_count = []
  soft_straight_count = []
  soft_breaking_count = []
  hard_straight_re288 = []
  hard_breaking_re288 = []
  soft_straight_re288 = []
  soft_breaking_re288 = []
  hard_straight_s_o = []
  hard_breaking_s_o = []
  soft_straight_s_o = []
  soft_breaking_s_o = []

  df = data[data['player_name']==i]

  for j in [2016, 2017, 2018, 2019]:
    df_year = df[df['game_year']==j]
    g_uniq = df_year['game_date'].unique()
    gnum = 1
    for k in g_uniq:
      game_data = df_year[df_year['game_date']==k]
      
      # Create dfs for each pitch type
      hard_straight = game_data.loc[game_data.pitch_name.isin(hrd_strt)]
      hard_breaking = game_data.loc[game_data.pitch_name.isin(hrd_brkng)]
      soft_straight = game_data.loc[game_data.pitch_name.isin(sft_strt)]
      soft_breaking = game_data.loc[game_data.pitch_name.isin(sft_brkng)]
      
      # Get the count of each pitch type per game
      hstrt = len(hard_straight)
      hbrk = len(hard_breaking)
      sstrt = len(soft_straight)
      sbrk = len(soft_breaking)
      
      # Get the sum of RE288 by pitch type for each game
      hstrt288 = hard_straight.RE288_change.sum()
      hbrk288  = hard_breaking.RE288_change.sum()
      sstrt288 = soft_straight.RE288_change.sum()
      sbrk288  = soft_breaking.RE288_change.sum()

      # Get the count of strikes/outs that followed each pitch type per game
      hstrt_strike = len(hard_straight.loc[hard_straight.description.isin(s_o)])
      hbrk_strike = len(hard_breaking.loc[hard_breaking.description.isin(s_o)])
      sstrt_strike = len(soft_straight.loc[soft_straight.description.isin(s_o)])
      sbrk_strike = len(soft_breaking.loc[soft_breaking.description.isin(s_o)])
      
      # Append the vals to the respective lists
      game_number.append(gnum)
      gnum += 1
      year.append(j)
      # Pitches
      hard_straight_count.append(hstrt)
      hard_breaking_count.append(hbrk)
      soft_straight_count.append(sstrt)
      soft_breaking_count.append(sbrk)
      # RE288
      hard_straight_re288 = list(hard_straight_re288)
      hard_breaking_re288 = list(hard_breaking_re288)
      soft_straight_re288 = list(soft_straight_re288)
      soft_breaking_re288 = list(soft_breaking_re288)
      hard_straight_re288.append(hstrt288)
      hard_breaking_re288.append(hbrk288)
      soft_straight_re288.append(sstrt288)
      soft_breaking_re288.append(sbrk288)
      # Strikes/outs
      hard_straight_s_o.append(hstrt_strike)
      hard_breaking_s_o.append(hbrk_strike)
      soft_straight_s_o.append(sstrt_strike)
      soft_breaking_s_o.append(sbrk_strike)

      # Correct for values so we don't end up taking the log of a negative number
      min_val = min(hard_straight_re288) + 0.01
      hard_straight_re288 = hard_straight_re288 + min_val
      hard_breaking_re288 = hard_breaking_re288 + min_val
      soft_straight_re288 = soft_straight_re288 + min_val
      soft_breaking_re288 = soft_breaking_re288 + min_val
        
    # Log pitch and outcome ratios using fastball as Bi
    logged_bx = []
    logged_RE288 = []
    logged_strikes = []

    for l in list(range(len(hard_straight_count))):
      behavior = (hard_straight_count[l]+1)/(hard_breaking_count[l] + soft_straight_count[l] + soft_breaking_count[l]+1)
      RE288 = (hard_straight_re288[l]+1)/(hard_breaking_re288[l] + soft_straight_re288[l] + soft_breaking_re288[l]+1)
      strikes = (hard_straight_s_o[l]+1)/(hard_breaking_s_o[l] + soft_straight_s_o[l] + soft_breaking_s_o[l]+1)
      behavior = math.log(behavior, 10)
      RE288 = math.log(RE288, 10)
      strikes = math.log(strikes, 10)
      logged_bx.append(behavior)
      logged_RE288.append(RE288)
      logged_strikes.append(strikes)
    
    # Combine the lists and save df for the pitcher
    count_data = pd.DataFrame({'Year':year, 'Game': game_number, \
                               'HStrt': hard_straight_count, 'HBrk':hard_breaking_count, \
                               'SStrt':soft_straight_count,'SBrk':soft_breaking_count, \
                               'HStrtRE288':hard_straight_re288, 'HBrkRE288':hard_breaking_re288, \
                               'SStrtRE288':soft_straight_re288, 'SBrkRE288':soft_straight_re288, \
                               'HStrtStrike':hard_straight_s_o, 'HBrkStrike':hard_breaking_s_o, \
                               'SStrtStrike':soft_straight_s_o, 'SBrkStrike':soft_breaking_s_o, \
                               'Logged_Bx':logged_bx, 'Logged_RE288':logged_RE288, \
                               'Logged_Strikes':logged_strikes})
    # Save the data
    count_data.to_csv('%s_counts.csv' %i)

    # Fit the GME and return the parameters
    b, s = np.polyfit()

    # Plot it
    
    # Save the data


In [ ]:
for i in list(range(len(hard_straight_count))):
  print("Game", i, hard_straight_count[i] + hard_breaking_count[i] + soft_straight_count[i] + soft_breaking_count[i])

In [18]:
data.description.unique()

array(['hit_into_play', 'ball', 'foul', 'hit_into_play_score',
       'swinging_strike', 'swinging_strike_blocked', 'called_strike',
       'hit_into_play_no_out', 'blocked_ball', 'intent_ball', 'foul_bunt',
       'hit_by_pitch', 'foul_tip', 'missed_bunt', 'pitchout',
       'swinging_pitchout', 'pitchout_hit_into_play_score',
       'foul_pitchout', 'bunt_foul_tip'], dtype=object)

In [16]:
data.head(10)

,pitch_type,game_date,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,fielder_2,effective_speed,game_pk,pitcher.1,fielder_2.1,at_bat_number,pitch_number,pitch_name,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,game_state,inning_same,total_runs,change_score,runs_inn_end,RE288,RE288_change
0,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,field_out,hit_into_play,5.0,Brock Holt lines out to left fielder Rajai Dav...,R,L,R,CLE,BOS,X,7.0,line_drive,3.0,1.0,2016,0.0,0.0,1.0,2.0,9.0,Top,543228.0,93.704,446867.0,545333.0,543228.0,74.0,5.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,65.0,False,8,0,0.0,1.969061,-0.016539
1,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,2.0,1.0,2016,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.288,446867.0,545333.0,543228.0,74.0,4.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,61.0,True,8,0,0.0,1.952521,0.015919
2,FT,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,12.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,1.0,1.0,2016,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.067,446867.0,545333.0,543228.0,74.0,3.0,2-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,57.0,True,8,0,0.0,1.968441,0.002399
3,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,0.0,1.0,2016,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.092,446867.0,545333.0,543228.0,74.0,2.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Strategic,53.0,True,8,0,0.0,1.970840,-0.030190
4,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,foul,9.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,0.0,0.0,2016,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.887,446867.0,545333.0,543228.0,74.0,1.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,49.0,True,8,0,2.0,1.940650,0.012106
5,FC,2016-04-05,Trevor Bauer,120074.0,545333.0,home_run,hit_into_play_score,14.0,David Ortiz homers (1) on a line drive to righ...,R,L,R,CLE,BOS,X,NaN,line_drive,2.0,2.0,2016,0.0,0.0,1.0,1.0,9.0,Top,543228.0,91.325,446867.0,545333.0,543228.0,71.0,5.0,Cutter,4.0,2.0,4.0,2.0,Infield shift,Standard,50.0,True,6,2,2.0,1.952757,-0.012106
6,FT,2016-04-05,Trevor Bauer,120074.0,545333.0,NaN,swinging_strike,14.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,2.0,1.0,2016,0.0,0.0,1.0,1.0,9.0,Top,543228.0,94.898,446867.0,545333.0,543228.0,71.0,4.0,2-Seam Fastball,4.0,2.0,4.0,2.0,Infield shift,Standard,49.0,True,6,0,2.0,1.940650,-0.006607
7,CU,2016-04-05,Trevor Bauer,543768.0,545333.0,strikeout,swinging_strike_blocked,14.0,Travis Shaw strikes out swinging.,R,L,R,CLE,BOS,S,2.0,NaN,0.0,2.0,2016,0.0,0.0,1.0,1.0,9.0,Top,543228.0,78.433,446867.0,545333.0,543228.0,73.0,3.0,Curveball,6.0,2.0,6.0,2.0,Standard,Standard,13.0,True,6,0,2.0,1.934044,0.100194
8,CH,2016-04-05,Trevor Bauer,120074.0,545333.0,NaN,ball,13.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,1.0,1.0,2016,0.0,0.0,1.0,1.0,9.0,Top,543228.0,87.757,446867.0,545333.0,543228.0,71.0,3.0,Changeup,4.0,2.0,4.0,2.0,Infield shift,Standard,46.0,True,6,0,2.0,2.034237,-0.117571
9,FC,2016-04-05,Trevor Bauer,543768.0,545333.0,NaN,foul,5.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,0.0,1.0,2016,0.0,0.0,1.0,1.0,9.0,Top,543228.0,89.788,446867.0,545333.0,543228.0,73.0,2.0,Cutter,6.0,2.0,6.0,2.0,Standard,Standard,59.0,True,6,0,2.0,1.916667,-0.007078


# Fit GME for all pitchers for each season using strikes/outs as reinforcer. 